In [1]:
from tensorflow import keras
from keras import layers

import pandas as pd
import numpy as np

2023-03-14 16:37:22.126755: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 16:37:30.784226: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/tyler/miniconda3/envs/tf/lib/:/home/tyler/miniconda3/envs/tf/lib/
2023-03-14 16:37:30.784315: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/tyler/miniconda3/envs/tf/lib/:/home/tyler/minicond

In [2]:
from utils import dataset

In [3]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

2023-03-14 16:37:33.412911: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-03-14 16:37:33.413211: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: tyler-Lenovo-Y520-15IKBN
2023-03-14 16:37:33.413244: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: tyler-Lenovo-Y520-15IKBN
2023-03-14 16:37:33.413539: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.85.5
2023-03-14 16:37:33.413649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.85.5
2023-03-14 16:37:33.413675: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 525.85.5
2023-03-14 16:37:33.414588: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized wit

In [11]:
# old model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.activations import relu
from keras.optimizers import Adam
from keras.losses import mean_squared_error
#import tensorflow as tf

model = keras.Sequential([
    #data_augmentation,
    keras.layers.Rescaling(1./255),
    Conv2D(3, (5,5), strides=2, padding="same", activation=relu, input_shape=(160,320,3)),
    #MaxPooling2D(2),
    Conv2D(24, (5, 5), strides=2, padding="same", activation=relu),
    #MaxPooling2D(2),
    Conv2D(36, (5, 5), strides=2, padding="same", activation=relu),
    #MaxPooling2D(2),
    Conv2D(48, (3, 3), strides=2, padding="same", activation=relu),
    #MaxPooling2D(2),
    Conv2D(64, (3, 3), strides=2, padding="same", activation=relu),
    #MaxPooling2D(2),
    #Conv2D(64, (3, 3), strides=2, padding="same", activation=relu),
    Dropout(0.5),
    Flatten(),
    #Dense(1164, activation=relu),
    Dense(100, activation=relu),
    Dense(50, activation=relu),
    Dense(10, activation=relu),
    Dense(1, activation="tanh"),

])

model.compile(loss=mean_squared_error, optimizer=Adam(), metrics=['mean_squared_error', 'mean_absolute_error'])


In [12]:
train_csv='/home/tyler/Desktop/testObjLab/autopilot_project/data_set/train_data/driving_log.csv'
batchSize = 32
train_ds=dataset(train_csv, batchsize=batchSize)

In [5]:
import pandas as pd
import tensorflow as tf
import os

# Set the directory for your images
image_dir = '/home/tyler/Desktop/testObjLab/autopilot_project/data_set/train_data/IMG/'
size = 2000
# Load the images into a list
X = []
for filename in os.listdir(image_dir)[:size]:
    img_path = os.path.join(image_dir, filename)
    img = tf.keras.preprocessing.image.load_img(
        img_path, target_size=(160,320)) #160, 320
    X.append(tf.keras.preprocessing.image.img_to_array(img))

# Convert the list of images to a NumPy array
X = tf.convert_to_tensor(X)




2023-03-14 13:07:09.151227: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1228800000 exceeds 10% of free system memory.


In [6]:
train_dataset_y = pd.read_csv('/home/tyler/Desktop/testObjLab/autopilot_project/data_set/train_data/driving_log.csv',
                              delimiter="\t", header=None, names=["Steering"])

Y = tf.convert_to_tensor(
    train_dataset_y["Steering"][:size].values, dtype=tf.float32)


ValueError: could not convert string to float: '/home/tyler/Desktop/testObjLab/autopilot_project/data_set/train_data/IMG/center_2023_03_14_12_44_13_550.jpg,/home/tyler/Desktop/testObjLab/autopilot_project/data_set/train_data/IMG/left_2023_03_14_12_44_13_550.jpg,/home/tyler/Desktop/testObjLab/autopilot_project/data_set/train_data/IMG/right_2023_03_14_12_44_13_550.jpg,0,0,0,2.104585'

In [7]:
# Create a dataset from the images and labels
#dataset = tf.data.Dataset.from_tensor_slices((X, Y))

# Shuffle and batch the dataset
#batch_size = 32
#dataset = dataset.shuffle(buffer_size=len(X))
#dataset = dataset.batch(batch_size)

# Use 80% of data for training, 20% for validation
split_index = int(0.8 * len(X))
X_train, X_val = X[:split_index], X[split_index:]
Y_train, Y_val = Y[:split_index], Y[split_index:]


In [9]:
X_train.shape

TensorShape([1600, 160, 300, 3])

In [13]:
#callbacks = [
#    keras.callbacks.ModelCheckpoint(
#        filepath="myModel.h5",
#        save_best_only=True,
#        monitor="val_loss")
#]
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="myModel.h5",
    )
]
history = model.fit(
    train_ds,
    epochs=20,
    callbacks=callbacks)


Epoch 1/20
181/181 [==============================] - 35s 191ms/step - loss: 0.0191 - mean_squared_error: 0.0191 - mean_absolute_error: 0.0581
Epoch 2/20
181/181 [==============================] - 35s 191ms/step - loss: 0.0191 - mean_squared_error: 0.0191 - mean_absolute_error: 0.0578
Epoch 3/20
181/181 [==============================] - 35s 194ms/step - loss: 0.0191 - mean_squared_error: 0.0191 - mean_absolute_error: 0.0572
Epoch 4/20
181/181 [==============================] - 35s 192ms/step - loss: 0.0191 - mean_squared_error: 0.0191 - mean_absolute_error: 0.0595
Epoch 5/20
181/181 [==============================] - 35s 191ms/step - loss: 0.0190 - mean_squared_error: 0.0190 - mean_absolute_error: 0.0579
Epoch 6/20
181/181 [==============================] - 35s 191ms/step - loss: 0.0191 - mean_squared_error: 0.0191 - mean_absolute_error: 0.0579
Epoch 7/20
181/181 [==============================] - 35s 191ms/step - loss: 0.0190 - mean_squared_error: 0.0190 - mean_absolute_error: 0.0567

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 200, 200, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 200, 200, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 100, 100, 3)       228       
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 24)        1824      
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 36)        21636     
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 48)        15600     
                                                                 
 conv2d_4 (Conv2D)           (None, 7, 7, 64)         

In [13]:
import matplotlib.pyplot as plt
accuracy = history.history["mean_squared_error"]
val_accuracy = history.history["val_mean_squared_error"]
loss = history.history["mean_absolute_error"]
val_loss = history.history["val_mean_absolute_error"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training")
plt.plot(epochs, val_accuracy, "b", label="Validation")
plt.title("Training and validation MSE")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training")
plt.plot(epochs, val_loss, "b", label="Validation")
plt.title("Training and validation MAE")
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'matplotlib'